# Visual SLAM Colab Setup (DROID-SLAM + 3DGS Mapping)

This notebook sets up the repo, installs dependencies, builds DROID-SLAM, downloads TUM, and runs the pipeline.

Update the repo URL below if needed.



In [ ]:
# --- Config ---
REPO_URL = "https://github.com/<YOUR_ORG>/visual-slam.git"  # TODO: replace
REPO_DIR = "/content/visual-slam"
DROID_DIR = "/content/DROID-SLAM"
DATASET_DIR = "/content/datasets/tum/fr3_office"

!nvidia-smi || true



In [ ]:
# --- System deps ---
!sudo apt-get update -y
!sudo apt-get install -y libeigen3-dev ninja-build



In [ ]:
# --- Clone repo ---
import os
import subprocess

if os.path.isdir(REPO_DIR):
    print("Repo already exists, skipping clone")
else:
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)

%cd {REPO_DIR}



In [ ]:
# --- Install Python deps ---
%pip install -r requirements.txt



In [ ]:
# --- Clone DROID-SLAM ---
import os
import subprocess

if os.path.isdir(DROID_DIR):
    print("DROID-SLAM already exists, skipping clone")
else:
    subprocess.run(["git", "clone", "https://github.com/princeton-vl/DROID-SLAM", DROID_DIR], check=True)

%cd {DROID_DIR}
!git submodule update --init --recursive



In [ ]:
# --- Install DROID-SLAM deps ---
import pathlib

req_path = pathlib.Path(DROID_DIR) / "requirements.txt"
if req_path.exists():
    lines = req_path.read_text().splitlines()
    lines = [ln for ln in lines if ln and not ln.lower().startswith("open3d")]
    tmp_req = "/tmp/droid_requirements.txt"
    pathlib.Path(tmp_req).write_text("\n".join(lines) + "\n")
    %pip install -r /tmp/droid_requirements.txt

# build third-party modules (requires torch)
%pip install -e thirdparty/lietorch --no-build-isolation
%pip install -e thirdparty/pytorch_scatter --no-build-isolation



In [ ]:
# --- Install DROID-SLAM ---
import os
import subprocess

env = os.environ.copy()
env["PIP_NO_BUILD_ISOLATION"] = "1"
env["EIGEN_INCLUDE_DIR"] = "/usr/include/eigen3"
env["CPLUS_INCLUDE_PATH"] = "/usr/include/eigen3:" + env.get("CPLUS_INCLUDE_PATH", "")
env["C_INCLUDE_PATH"] = "/usr/include/eigen3:" + env.get("C_INCLUDE_PATH", "")

subprocess.run(["pip", "install", "-e", ".", "--no-build-isolation"], check=True, env=env)



In [ ]:
# --- Download DROID-SLAM weights ---
import os
import subprocess

script = os.path.join(DROID_DIR, "tools", "download_model.sh")
if os.path.exists(script):
    subprocess.run(["bash", script], check=True)
else:
    print("download_model.sh not found; check DROID-SLAM README")



In [ ]:
# --- Configure config for Colab ---
import yaml
from pathlib import Path

cfg_path = Path(REPO_DIR) / "configs" / "tum_fr3_office.yaml"
cfg = yaml.safe_load(cfg_path.read_text())

cfg["dataset"]["root"] = DATASET_DIR
cfg["tracking"]["mode"] = "droid"
cfg["tracking"]["use_gt_init"] = False
cfg.setdefault("tracking", {}).setdefault("droid", {})
cfg["tracking"]["droid"]["repo_path"] = DROID_DIR

cfg_path.write_text(yaml.safe_dump(cfg, sort_keys=False))
print(cfg_path.read_text())



In [ ]:
# --- Run SLAM (will download dataset if missing) ---
import subprocess

subprocess.run([
    "python",
    "run_slam.py",
    "--config",
    str(Path(REPO_DIR) / "configs" / "tum_fr3_office.yaml"),
], check=True)



In [ ]:
# --- Visualize final PLY ---
import subprocess

subprocess.run([
    "python",
    "view_ply.py",
    "--ply",
    str(Path(REPO_DIR) / "outputs" / "map.ply"),
], check=True)

